# Введение в глубинное обучение, ФКН ВШЭ

## Домашнее задание 3. Обработка текстов.

### Общая информация

Дата выдачи: 13.01.2022

Мягкий дедлайн: 23:59MSK 6.02.2022

Жесткий дедлайн: 23:59MSK 10.02.2022

Оценка после штрафа после мягкого дедлайна вычисляется по формуле $M_{penalty} = M_{full} \cdot 0.85^{t/1440}$, где $M_{full}$ — полная оценка за работу без учета штрафа, а $t$ — время в минутах, прошедшее после мягкого дедлайна (округление до двух цифр после запятой). Таким образом, спустя первые сутки после мягкого дедлайна вы не можете получить оценку выше 8.5, а если сдать перед самым жестким дедлайном, то ваш максимум — 5.22 балла.

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В данном домашнем задании вам предстоит предсказывать пользовательскую оценку отеля по тексту отзыва. Нужно обучиться на данных с кэггла и заслать в [соревнование](https://www.kaggle.com/t/3e8fa6cec6d048bf8e93fb72e441d88c) предикт. По той же ссылке можете скачать данные.

Мы собрали для вас отзывы по 1500 отелям из совершенно разных уголков мира. Что это за отели - секрет. Вам дан текст отзыва и пользовательская оценка отеля. Ваша задача - научиться предсказывать оценку отеля по отзыву.

Главная метрика - Mean Absolute Error (MAE). Во всех частях домашней работы вам нужно получить значение MAE не превышающее 0.92 на публичном лидерборде. В противном случае мы будем вынуждены не засчитать задание :( 

#### Про данные:
Каждое ревью состоит из двух текстов: positive и negative - плюсы и минусы отеля. В столбце score находится оценка пользователя - вещественное число 0 до 10. Вам нужно извлечь признаки из этих текстов и предсказать по ним оценку.

Для локального тестирования используйте предоставленное разбиение на трейн и тест.

Good luck & have fun! 💪

In [1]:
import torch
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.display import clear_output
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import Vocab
from collections import Counter

import torchtext

#### Использовать любые данные для обучения кроме предоставленных организаторами строго запрещено. В последней части можно использовать предобученные модели из библиотеки `transformers`.

In [2]:
PATH_TO_TRAIN_DATA = 'data/train.csv'

In [3]:
import pandas as pd

df = pd.read_csv(PATH_TO_TRAIN_DATA)
df.sample(10)

,review_id,negative,positive,score
30470,4e7190aa04bf9d16404b0542b0497c2e,Quite expensive,Really helpful polite staff Good quality d co...,9.6
40627,6839cf76a9497590d2c08d7bdd420e86,Nothing at all,Location is unbeatable in walkable distance a...,10.0
19050,31642d36322487ae0db4ca12e30618f0,Nothing,The room was great and the staff was lovely s...,10.0
75833,c23d6eb9f80f40090c53018afc5be86f,ceiling height compromised on floor 6 overloo...,great staff,7.9
11050,1ca844ff439b9c966b5e11609ba660a2,The breakfast no variety of cheeses mainly it...,Location the room lunch in the room,9.6
85176,d9c124e0a6cc4aeef21b036b871395c8,No Negative,Everything the hotel is pur class,9.6
88925,e3631610790276e7d842c5eb47339efc,No Negative,Beds extremely comfortable Staff were very we...,9.2
69829,b2eae52da76992a1808c6deb9c88e201,Only Spanish TV channels reception under char...,Easy to find close to shops and 2 major tube ...,5.8
77036,c5391e0c528002eaea481ee9477ef7db,Size of the room was small,The hotel was in a good location but the room...,8.3
39941,667f1cbaba1333c8fe7f39261d9bac8b,a little bit expensive but not a big gap comp...,The food at breakfast was really good a lot o...,9.6


Предобработка текста может сказываться на качестве вашей модели.
Сделаем небольшой препроцессинг текстов: удалим знаки препинания, приведем все слова к нижнему регистру. 
Однако можно не ограничиваться этим набором преобразований. Подумайте, что еще можно сделать с текстами, чтобы помочь будущим моделям? Добавьте преобразования, которые могли бы помочь по вашему мнению.

In [4]:
# df[df['positive'] == ' ']
# Имеем смысл удалить строки, где нет информации по p/n

Также мы добавили разбиение текстов на токены. Теперь каждая строка-ревью стала массивом токенов.

In [5]:
import string

import nltk
# nltk.download('punkt')

from nltk.tokenize import word_tokenize

def process_text(text):
    return [word for word in word_tokenize(text.lower()) if word not in string.punctuation]

# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

def process_lower(text):
    return text.lower()
#     return lemmatizer.lemmatize(text.lower())

In [6]:
s = "Now bats are leaving their trees, They're joining the call, Seven Satanic Hell Preachers Heading for the hall. \
Bringing a blood of a newborn child, Got to succeed, if not it's Satan's fall"
lemmatizer = WordNetLemmatizer()

# import spacy
# nlp = spacy.load('en', disable=['parser', 'ner'])
# doc = nlp(s)
# print(" ".join([token.lemma_ for token in doc]))

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# nltk.download('stopwords')
 
# stop_words = set(stopwords.words('english') + [ "'s", "'re"]) - {'no'}
stop_words = set(['a', 'an', 'the', "'s", 'to', "'re", 'is', 'are', 'be', 'been', 'was', 'were', 'has', 'had', 'have'])
def process_text_advanced(text):
    text = [lemmatizer.lemmatize(word) for word in word_tokenize(text.lower()) 
            if word not in string.punctuation and word not in stop_words]
    return ' '.join(text)
process_text_advanced(s)

'now bat leaving their tree they joining call seven satanic hell preacher heading for hall bringing blood of newborn child got succeed if not it satan fall'

In [7]:
# df['negative'] = df['negative'].apply(process_text)
# df['positive'] = df['positive'].apply(process_text)

df['negative'] = df['negative'].apply(process_text_advanced)
df['positive'] = df['positive'].apply(process_text_advanced)

In [8]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, random_state=1412) # <- для локального тестирования
y_train = df_train['score'].to_numpy()
y_test = df_test['score'].to_numpy()

### Часть 1. 1 балл

Обучите логистическую или линейную регрессию на TF-IDF векторах текстов.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error as MAE

Предскажите этой моделью тестовые данные из [соревнования](https://www.kaggle.com/t/3e8fa6cec6d048bf8e93fb72e441d88c) и сделайте сабмит. Какой у вас получился скор? Прикрепите скриншот из кэггла.

In [10]:
# data_train = (df_train['negative'] + ' ' + df_train['positive']).tolist()
# data_test = (df_test['negative'] + ' ' + df_test['positive']).tolist()
# Вместо этого мне посоветовали делать так:

In [11]:
def pdsp(data):
    return pd.DataFrame.sparse.from_spmatrix(data)

def transform_fragments(v1, v2, data: list, mode=1):
    '''
        mode = 1: fitting + transform, input - train / +test data
        mode = 2: transform, input - only train or test data
    '''
    if mode == 2:
        assert (len(data) == 1)
    
    return_data = []
    for ind, d in enumerate(data):
        if ind > 0 or mode == 2:
            data_pos = v1.transform(d['positive'])
            data_neg = v2.transform(d['negative'])
        else:
            data_pos = v1.fit_transform(d['positive'])
            data_neg = v2.fit_transform(d['negative'])
            
        data_ = pd.concat([pdsp(data_pos), pdsp(data_neg)], axis=1, ignore_index=True)
        return_data += [data_.sparse.to_coo().tocsr()]
    
    return return_data, v1, v2

In [12]:
vec1, vec2 = TfidfVectorizer(), TfidfVectorizer()
X, vec1, vec2 = transform_fragments(vec1, vec2, [df_train, df_test])
X_train, X_test = X[0], X[1]
assert X_train.shape[1] == X_test.shape[1]

In [13]:
# Выдавало ошибку, поэтому перевел в 100-балльную
logreg = LogisticRegression(n_jobs=-1).fit(X_train, (y_train * 10).astype(int))
rig = Ridge().fit(X_train, y_train)
lasso = Lasso().fit(X_train, y_train)

In [14]:
y_pred_rig = rig.predict(X_test)
y_pred_lass = lasso.predict(X_test)
y_pred_lr = logreg.predict(X_test)

'''   
    MAE for Ridge: 0.8440459199291432
    MAE for Lasso: 1.3166713036799997
    MAE for LogReg: 0.917024
'''

print('MAE for Ridge:', MAE(y_test, y_pred_rig))
print('MAE for Lasso:', MAE(y_test, y_pred_lass))
print('MAE for LogReg:', MAE(y_test, y_pred_lr / 10))

MAE for Ridge: 0.8316101258538141
MAE for Lasso: 1.3166713036799997
MAE for LogReg: 0.90306


`Ridge`-регрессия справилась лучше всех.

In [15]:
PATH_TO_TEST_DATA = 'data/test.csv'
for_submit_df = pd.read_csv(PATH_TO_TEST_DATA)
for_submit_df['negative'] = for_submit_df['negative'].apply(process_text_advanced)
for_submit_df['positive'] = for_submit_df['positive'].apply(process_text_advanced)

X_subm, _, _ = transform_fragments(vec1, vec2, [for_submit_df], mode=2)

y_pred_rig_subm = rig.predict(X_subm[0])
submit = for_submit_df.drop(columns=['negative', 'positive'])
submit['score'] = y_pred_rig_subm

In [16]:
submit.to_csv('sumbit.csv', index=False)

### Часть 2. 2 балла

Обучите логистическую или линейную регрессию на усредненных Word2Vec векторах. 

In [17]:
# !pip install gensim
import gensim

from gensim.models import Word2Vec

In [18]:
df_train_new = df_train.copy()
df_test_new = df_test.copy()
df_train_new['negative'] = df_train_new['negative'].apply(process_text)
df_train_new['positive'] = df_train_new['positive'].apply(process_text)
df_test_new['negative'] = df_test_new['negative'].apply(process_text)
df_test_new['positive'] = df_test_new['positive'].apply(process_text)

In [19]:
w2v_model_pos = Word2Vec(df_train_new['positive'])
w2v_model_neg = Word2Vec(df_train_new['negative'])

In [20]:
import numpy as np
from scipy.sparse.csr import csr_matrix
# Позаимстовано из https://habr.com/ru/company/ods/blog/329410/
class mean_vectorizer():
    def __init__(self, w2v_model):
        self.w2v_model = w2v_model

    def fit(self, X):
        self.w2v_dict = dict(zip(self.w2v_model.wv.index_to_key, self.w2v_model.wv.vectors))
        self.dim = self.w2v_model.wv.vectors.shape[1]
        return self 

    def transform(self, X):
        return csr_matrix([
            np.mean([self.w2v_dict[w] for w in words if w in self.w2v_dict] 
                or [np.zeros(self.dim)], axis=0) for words in X
        ])
    
    def fit_transform(self, X):
        self = self.fit(X)
        return self.transform(X)

In [21]:
vec1, vec2 = mean_vectorizer(w2v_model_pos), mean_vectorizer(w2v_model_neg)
X, vec1, vec2 = transform_fragments(vec1, vec2, [df_train_new, df_test_new])

In [22]:
def get_quality(X_train, y_train, X_test, y_test, alert=False):
    rig = Ridge().fit(X_train, y_train)
    y_pred_rig = rig.predict(X_test)
    q_rig = MAE(y_test, y_pred_rig)
    
    if alert:
        print('MAE for Ridge:', q_rig)
        
    return q_rig

In [23]:
_ = get_quality(X[0], y_train, X[1], y_test, alert=True)

MAE for Ridge: 0.9610207923484096


Усредняя w2v вектора, мы предполагаем, что каждое слово имеет равноценный вклад в смысл предложения, однако это может быть не совсем так. Теперь попробуйте воспользоваться другой концепцией и перевзвесить слова при получении итогового эмбеддинга текста. В качестве весов используйте IDF (Inverse document frequency)

In [24]:
from collections import defaultdict

class tfidf_vectorizer():
    def __init__(self, w2v_model):
        self.w2v_model = w2v_model
        self.word2weight = None

    def fit(self, X):
        self.w2v_dict = dict(zip(self.w2v_model.wv.index_to_key, self.w2v_model.wv.vectors))
        self.dim = self.w2v_model.wv.vectors.shape[1]
        
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf = tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        )

        return self

    def transform(self, X):
        return csr_matrix([
                np.mean([self.w2v_dict[w] * self.word2weight[w]
                         for w in words if w in self.w2v_dict] or
                        [np.zeros(self.dim)], axis=0) for words in X
            ])

    def fit_transform(self, X):
        self = self.fit(X)
        return self.transform(X)

In [25]:
vec1, vec2 = tfidf_vectorizer(w2v_model_pos), tfidf_vectorizer(w2v_model_neg)
X, vec1, vec2 = transform_fragments(vec1, vec2, [df_train_new, df_test_new])

In [26]:
_ = get_quality(X[0], y_train, X[1], y_test, alert=True)

MAE for Ridge: 0.9642303949205574


Проведите эксперименты с размерностью эмбеддинга. Для каждого из двух методов постройте график зависимости качества модели от размерности эмбеддинга. 

In [ ]:
emb_dims = [i * 50 for i in range(4, 11)]
results = {
    'mean': [],
    'tfidf': []
}

for dim in tqdm(emb_dims):
    
    w2v_model_pos = Word2Vec(df_train_new['positive'], window=30, vector_size=dim)
    w2v_model_neg = Word2Vec(df_train_new['negative'], window=30, vector_size=dim)
    
    vec1, vec2 = mean_vectorizer(w2v_model_pos), mean_vectorizer(w2v_model_neg)
    X_mean, _, _ = transform_fragments(vec1, vec2, [df_train_new, df_test_new])
    mv_q_rig = get_quality(X_mean[0], y_train, X_mean[1], y_test)
    
    vec1, vec2 = tfidf_vectorizer(w2v_model_pos), tfidf_vectorizer(w2v_model_neg)
    X_idf, _, _ = transform_fragments(vec1, vec2, [df_train_new, df_test_new])
    tfidf_q_rig = get_quality(X_idf[0], y_train, X_idf[1], y_test)
    
    results['mean'].append(mv_q_rig)
    results['tfidf'].append(tfidf_q_rig)

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(16, 5))
ax1.plot(emb_dims, results['mean'], color='lime', label='mean')
ax1.plot(emb_dims, results['tfidf'], color='blue', label='IDF mean')
ax1.set_xlabel('embegging size')
ax1.set_ylabel('MAE')
ax1.set_title('Embegging size - MAE')
ax1.legend(shadow=False, fontsize=14)
plt.show()

In [ ]:
results['tfidf'][np.argmin(results['tfidf'])], emb_dims[np.argmin(results['tfidf'])], \
results['mean'][np.argmin(results['mean'])], emb_dims[np.argmin(results['mean'])]

#### Сделайте выводы:
Поначалу с ростом размера эмбеддинга качество улучшается, затем начинает скакать и вполне может уменьшаться и дальше. Это зависит от гиперпараметров. Но в целом по моим наблюдениям размер от 300 до 500 кажется вполне оптимальным.

Теперь попробуйте обучить логистическую или линейную регрессию на любых других эмбеддингах размерности 300 и сравните качество с Word2Vec.

In [ ]:
from gensim.models import FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument

model_ft_pos = FastText(df_train_new['positive'], window=30, vector_size=300)
model_ft_neg = FastText(df_train_new['negative'], window=30, vector_size=300)

documents_pos = [TaggedDocument(doc, [i]) for i, doc in enumerate(df_train_new['positive'])]
documents_neg = [TaggedDocument(doc, [i]) for i, doc in enumerate(df_train_new['negative'])]
model_d2v_pos = Doc2Vec(documents_pos, window=30, vector_size=300)
model_d2v_neg = Doc2Vec(documents_neg, window=30, vector_size=300)

In [ ]:
vec1, vec2 = tfidf_vectorizer(model_ft_pos), tfidf_vectorizer(model_ft_neg)
X, _, _ = transform_fragments(vec1, vec2, [df_train_new, df_test_new])
_ = get_quality(X[0], y_train, X[1], y_test, alert=True)

In [ ]:
vec1, vec2 = tfidf_vectorizer(model_d2v_pos), tfidf_vectorizer(model_d2v_neg)
X, _, _ = transform_fragments(vec1, vec2, [df_train_new, df_test_new])
_ = get_quality(X[0], y_train, X[1], y_test, alert=True)

#### Выводы:
Не лучше, чем word2vec + Ridge + IGF-взвешивание. Если сравнивать эти два метода, то из них лучше себя показывает IDF-взвешивание + Ridge + doc2vec.

Предскажите вашей лучшей моделью из этого задания тестовые данные из [соревнования](https://www.kaggle.com/t/3e8fa6cec6d048bf8e93fb72e441d88c) и сделайте сабмит. Какой у вас получился скор? Прикрепите скриншот из кэггла.

In [ ]:
assert

In [ ]:
vectorizer = tfidf_vectorizer(model_d2v)
X_train_all = vectorizer.fit_transform(df_train_all)
X_subm = vectorizer.transform(subm)

model = Ridge().fit(X_train_all, df['score'].to_numpy())
y_pred_rig_subm = model.predict(X_subm)
submit = for_submit_df.drop(columns=['negative', 'positive'])
submit['score'] = y_pred_rig_subm

submit.to_csv('sumbit_2.csv', index=False)

### Часть 3. 4 балла

Теперь давайте воспользуемся более продвинутыми методами обработки текстовых данных, которые мы проходили в нашем курсе. Обучите RNN/Transformer для предсказания пользовательской оценки.

Если будете обучать RNN, попробуйте ограничить максимальную длину предложения. Некоторые отзывы могут быть слишком длинные относительно остальных.

Чтобы пользоваться DataLoader, все его элементы должны быть одинаковой размерности. Для этого вы можете добавить нулевой паддинг ко всем предложениям (см пример pad_sequence)

In [ ]:
# data_train = (df_train['negative'] + ' ' + df_train['positive']).tolist()
# data_test = (df_test['negative'] + ' ' + df_test['positive']).tolist()

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
WORDS = set()
for sent in list(df['positive']):
    for w in sent:
        WORDS.add(w)
        
for sent in list(df['negative']):
    for w in sent:
        WORDS.add(w)

In [ ]:
int2word = dict(enumerate(tuple(WORDS)))
word2int = {w: ii for ii, w in int2word.items()}

In [ ]:
MAX_LEN = max(max(df['positive'].apply(len)), max(df['negative'].apply(len)))
MAX_LEN

### Следующий код будет во многом позаимствован с семинара во избежание головной боли

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

def dataset_iterator(texts):
    for text in texts:
        yield text.split()

In [ ]:
vocab = build_vocab_from_iterator(
    dataset_iterator(data_train),
                                    # г/п
    specials=['<pad>', '<unk>'], min_freq=10,
)

In [ ]:
def tokenize_sentences(vocab, data):
    tokens = []
    for text in dataset_iterator(data):
        sentence_tokens = [vocab[word] if word in vocab else vocab['<unk>'] for word in text]
        tokens += [sentence_tokens]
    return tokens

train_tokens = tokenize_sentences(vocab, data_train)
test_tokens = tokenize_sentences(vocab, data_test)

In [ ]:
import seaborn as sns
plt.rcParams.update({'font.size': 14})
sns.set_style('whitegrid')

lengths = np.array([len(tokens) for tokens in train_tokens])
sns.displot(lengths)
plt.show()

Тут можно было бы удалить те отзывы, где в полях пусто, или же с заполнениями а-ля `no positive`/`no negative`. Люди склонны ничего не писать и ставить высокие оценки, либо же писать `no positive` и ставить 0 или, напротив, писать `no negative` и ставить 10. Поэтому я решил ничего не удалять - такие данные могут сильно влиять на результат.

In [ ]:
max_length = 400
def get_data(data):
    tokenized_data = torch.full((len(data), max_length), vocab['<pad>'], dtype=torch.int32)
    for i, tokens in enumerate(data):
        length = min(max_length, len(tokens))
        tokenized_data[i, :length] = torch.tensor(tokens[:length])
    return tokenized_data
    
tokenized_train = get_data(train_tokens)
tokenized_test = get_data(test_tokens)
    
targets_train = torch.tensor(y_train, dtype=torch.int32)
targets_test = torch.tensor(y_test, dtype=torch.int32)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(tokenized_train, targets_train)
test_dataset = TensorDataset(tokenized_test, targets_test)

# г/п
batch_size = 100
train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=4, pin_memory=True)

### Transformer

### RNN

In [ ]:
from torch.nn.utils.rnn import pad_sequence

train_pos_pad = pad_sequence([torch.as_tensor([word2int[w] for w in seq][:MAX_LEN]) for seq in df_train['positive']], 
                           batch_first=True)

In [ ]:
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        ## TODO
        pass
        
    def __len__(self):
        ## TODO
        pass
    
    def __getitem__(self, idx):
        ## TODO
        pass

In [ ]:
BATCH_SIZE = 1

train_dataset = ReviewsDataset(df_train)
test_dataset = ReviewsDataset(df_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
NUM_EPOCHS = 1

for n in range(NUM_EPOCHS):
    model.train()
    ## TODO

### Контест (до 3 баллов)

По итогам всех ваших экспериментов выберите модель, которую считаете лучшей. Сделайте сабмит в контест. В зависимости от вашего скора на публичном лидерборде, мы начислим вам баллы:

 - <0.76 - 3 балла
 - [0.76; 0.78) - 2 балла
 - [0.78; 0.8) - 1 балл